## Load Libs

In [1]:
import sys

sys.path.append('../input/monai-v081/')

In [2]:
import gc
from glob import glob
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
from monai.inferers import sliding_window_inference
from monai.data import decollate_batch
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import json

In [3]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    anonymous = "must"
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

To use your W&B account,
Go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. 
Get your W&B access token from here: https://wandb.ai/authorize


In [4]:
from monai.data import CacheDataset, DataLoader
from monai.transforms import (
    Compose,
    Activations,
    AsDiscrete,
    Activationsd,
    AsDiscreted,
    KeepLargestConnectedComponentd,
    Invertd,
    LoadImage,
    Transposed,
    LoadImaged,
    AddChanneld,
    CastToTyped,
    Lambdad,
    Resized,
    EnsureTyped,
    SpatialPadd,
    EnsureChannelFirstd,
)

## Prepare meta info.

### Thanks awsaf49, this section refers to:
https://www.kaggle.com/code/awsaf49/uwmgi-2-5d-infer-pytorch

In [5]:
def get_metadata(row):
    data = row['id'].split('_')
    case = int(data[0].replace('case',''))
    day = int(data[1].replace('day',''))
    slice_ = int(data[-1])
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

def path2info(row):
    path = row['image_path']
    data = path.split('/')
    slice_ = int(data[-1].split('_')[1])
    case = int(data[-3].split('_')[0].replace('case',''))
    day = int(data[-3].split('_')[1].replace('day',''))
    width = int(data[-1].split('_')[2])
    height = int(data[-1].split('_')[3])
    row['height'] = height
    row['width'] = width
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

In [6]:
sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
if not len(sub_df):
    debug = True
    sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')[:1000*3]
    sub_df = sub_df.drop(columns=['class','segmentation']).drop_duplicates()
else:
    debug = False
    sub_df = sub_df.drop(columns=['class','predicted']).drop_duplicates()
sub_df = sub_df.apply(lambda x: get_metadata(x),axis=1)

In [7]:
if debug:
    paths = glob(f'/kaggle/input/uw-madison-gi-tract-image-segmentation/train/**/*png',recursive=True)
#     paths = sorted(paths)
else:
    paths = glob(f'/kaggle/input/uw-madison-gi-tract-image-segmentation/test/**/*png',recursive=True)
#     paths = sorted(paths)
path_df = pd.DataFrame(paths, columns=['image_path'])
path_df = path_df.apply(lambda x: path2info(x),axis=1)
path_df.head()

,image_path,height,width,case,day,slice
0,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266,36,14,6
1,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266,36,14,82
2,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266,36,14,113
3,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266,36,14,76
4,/kaggle/input/uw-madison-gi-tract-image-segmen...,266,266,36,14,125


## Produce 3d data list for MONAI DataSet

In [8]:
test_df = sub_df.merge(path_df, on=['case','day','slice'], how='left')
test_df["case_id_str"] = test_df["id"].apply(lambda x: x.split("_", 2)[0])
test_df["day_num_str"] = test_df["id"].apply(lambda x: x.split("_", 2)[1])
test_df["slice_id"] = test_df["id"].apply(lambda x: x.split("_", 2)[2])

In [9]:
test_data = []

for group in test_df.groupby(["case_id_str", "day_num_str"]):

    case_id_str, day_num_str = group[0]
    group_id = case_id_str + "_" + day_num_str
    group_df = group[1].sort_values("slice_id", ascending=True)
    n_slices = group_df.shape[0]
    group_slices, group_ids = [], []
    for idx in range(n_slices):
        slc = group_df.iloc[idx]
        group_slices.append(slc.image_path)
        group_ids.append(slc.id)
    test_data.append({"image": group_slices, "id": group_ids})

## Prepare Transforms, Dataset, DataLoader

In [10]:
class cfg:
    img_size = (224, 224, 80)
    in_channels = 1
    out_channels = 4
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    weights = glob("../input/uw3dweights/unet_fold0_0.8789.pth")
    batch_size = 1
    sw_batch_size = 4

In [11]:
for i in cfg.weights:
    print(i)

../input/uw3dweights/unet_fold0_0.8789.pth


In [12]:
test_transforms = Compose(
    [
        LoadImaged(keys="image"), # d, h, w
        AddChanneld(keys="image"), # c, d, h, w
        Transposed(keys="image", indices=[0, 2, 3, 1]), # c, w, h, d
        Lambdad(keys="image", func=lambda x: x / x.max()),
#         SpatialPadd(keys="image", spatial_size=cfg.img_size),  # in case less than 80 slices
        EnsureTyped(keys="image", dtype=torch.float32),
    ]
)

test_ds = CacheDataset(
        data=test_data,
        transform=test_transforms,
        cache_rate=0.0,
        num_workers=2,
    )

test_dataloader = DataLoader(
    test_ds,
    batch_size=cfg.batch_size,
    num_workers=2,
    pin_memory=True,
)

## Prepare Network

In [13]:
model = UNet(
    spatial_dims=3,
    in_channels=cfg.in_channels,
    out_channels=cfg.out_channels,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    kernel_size=3,
    up_kernel_size=3,
    num_res_units=2,
    act="PRELU",
    norm="BATCH",
    dropout=0.2,
    bias=True,
    dimensions=None,
).to(cfg.device)

## Infer

In [14]:
# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    """ TBD
    
    Args:
        img (np.array): 
            - 1 indicating mask
            - 0 indicating background
    
    Returns: 
        run length as string formated
    """
    
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [15]:
outputs = []

post_pred = Compose([
    Activations(sigmoid=True),
    AsDiscrete(threshold=0.5)
])

model.eval()
torch.set_grad_enabled(False)
progress_bar = tqdm(range(len(test_dataloader)))
val_it = iter(test_dataloader)
for itr in progress_bar:
    batch = next(val_it)
    test_inputs = batch["image"].to(cfg.device)

    pred_all = []
    for weights in cfg.weights:
        model.load_state_dict(torch.load(weights))
        pred = sliding_window_inference(test_inputs, cfg.img_size, cfg.sw_batch_size, model)
        pred_all.append(pred)
        # do 4 tta
        for dims in [[2], [3], [2, 3]]:
            flip_pred = sliding_window_inference(torch.flip(test_inputs, dims=dims), cfg.img_size, cfg.sw_batch_size, model)
            flip_pred = torch.flip(flip_pred, dims=dims)
            pred_all.append(flip_pred)
    pred_all = torch.mean(torch.stack(pred_all), dim=0)[0]
    pred_all = post_pred(pred_all)[1:]
    # c, w, h, d to d, c, h, w
    pred_all = torch.permute(pred_all, [3, 0, 2, 1]).cpu().numpy().astype(np.uint8)
    id_outputs = from_engine(["id"])(batch)[0]

    for test_output, id_output in zip(pred_all, id_outputs):
        id_name = id_output[0]
        lb, sb, st = test_output
        outputs.append([id_name, "large_bowel", lb])
        outputs.append([id_name, "small_bowel", sb])
        outputs.append([id_name, "stomach", st])
    


100%|██████████| 7/7 [00:21<00:00,  3.06s/it]


In [16]:
  class cfg:
    img_size = (224, 224, 80)
    in_channels = 1
    out_channels = 3
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    weights = glob("../input/uw3dweights/large_unet_fold0_0.9024.pth")
    batch_size = 1
    sw_batch_size = 4

In [17]:
test_transforms = Compose(
    [
        LoadImaged(keys="image"), # d, h, w
        AddChanneld(keys="image"), # c, d, h, w
        Transposed(keys="image", indices=[0, 2, 3, 1]), # c, w, h, d
        Lambdad(keys="image", func=lambda x: x / x.max()),
#         SpatialPadd(keys="image", spatial_size=cfg.img_size),  # in case less than 80 slices
        EnsureTyped(keys="image", dtype=torch.float32),
    ]
)

test_ds = CacheDataset(
        data=test_data,
        transform=test_transforms,
        cache_rate=0.0,
        num_workers=2,
    )

test_dataloader = DataLoader(
    test_ds,
    batch_size=cfg.batch_size,
    num_workers=2,
    pin_memory=True,
)

In [18]:
model = UNet(
    spatial_dims=3,
    in_channels=cfg.in_channels,
    out_channels=cfg.out_channels,
    channels=(32, 64, 128, 256, 512),
    strides=(2, 2, 2, 2),
    kernel_size=3,
    up_kernel_size=3,
    num_res_units=2,
    act="PRELU",
    norm="BATCH",
    dropout=0.2,
    bias=True,
    dimensions=None,
).to(cfg.device)

In [19]:
outputs2 = []

post_pred = Compose([
    Activations(sigmoid=True),
    AsDiscrete(threshold=0.5)
])

model.eval()
torch.set_grad_enabled(False)
progress_bar = tqdm(range(len(test_dataloader)))
val_it = iter(test_dataloader)
for itr in progress_bar:
    batch = next(val_it)
    test_inputs = batch["image"].to(cfg.device)

    pred_all = []
    for weights in cfg.weights:
        model.load_state_dict(torch.load(weights))
        pred = sliding_window_inference(test_inputs, cfg.img_size, cfg.sw_batch_size, model)
        pred_all.append(pred)
        # do 4 tta
        for dims in [[2], [3], [2, 3]]:
            flip_pred = sliding_window_inference(torch.flip(test_inputs, dims=dims), cfg.img_size, cfg.sw_batch_size, model)
            flip_pred = torch.flip(flip_pred, dims=dims)
            pred_all.append(flip_pred)
    pred_all = torch.mean(torch.stack(pred_all), dim=0)[0]
    pred_all = post_pred(pred_all)
    # c, w, h, d to d, c, h, w
    pred_all = torch.permute(pred_all, [3, 0, 2, 1]).cpu().numpy().astype(np.uint8)
    id_outputs2 = from_engine(["id"])(batch)[0]

    for test_output2, id_output2 in zip(pred_all, id_outputs2):
        id_name = id_output2[0]
        lb, sb, st = test_output2
        outputs2.append([id_name, "large_bowel", lb])
        outputs2.append([id_name, "small_bowel", sb])
        outputs2.append([id_name, "stomach", st])
    


100%|██████████| 7/7 [00:22<00:00,  3.24s/it]


In [54]:
submit1 = pd.DataFrame(data=np.array(outputs), columns=["id", "class1", "predicted1"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [55]:
submit2 = pd.DataFrame(data=np.array(outputs2), columns=["id", "class2", "predicted2"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [22]:
  class cfg:
    img_size = (224, 224, 80)
    in_channels = 1
    out_channels = 4
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    weights = glob("../input/uw3dweights/unet_fold0_0.8932.pth")
    batch_size = 1
    sw_batch_size = 4

In [23]:
test_transforms = Compose(
    [
        LoadImaged(keys="image"), # d, h, w
        AddChanneld(keys="image"), # c, d, h, w
        Transposed(keys="image", indices=[0, 2, 3, 1]), # c, w, h, d
        Lambdad(keys="image", func=lambda x: x / x.max()),
#         SpatialPadd(keys="image", spatial_size=cfg.img_size),  # in case less than 80 slices
        EnsureTyped(keys="image", dtype=torch.float32),
    ]
)

test_ds = CacheDataset(
        data=test_data,
        transform=test_transforms,
        cache_rate=0.0,
        num_workers=2,
    )

test_dataloader = DataLoader(
    test_ds,
    batch_size=cfg.batch_size,
    num_workers=2,
    pin_memory=True,
)

In [24]:
model = UNet(
    spatial_dims=3,
    in_channels=cfg.in_channels,
    out_channels=cfg.out_channels,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    kernel_size=3,
    up_kernel_size=3,
    num_res_units=2,
    act="PRELU",
    norm="BATCH",
    dropout=0.2,
    bias=True,
    dimensions=None,
).to(cfg.device)

In [25]:
outputs3 = []

post_pred = Compose([
    Activations(sigmoid=True),
    AsDiscrete(threshold=0.5)
])

model.eval()
torch.set_grad_enabled(False)
progress_bar = tqdm(range(len(test_dataloader)))
val_it = iter(test_dataloader)
for itr in progress_bar:
    batch = next(val_it)
    test_inputs = batch["image"].to(cfg.device)

    pred_all = []
    for weights in cfg.weights:
        model.load_state_dict(torch.load(weights))
        pred = sliding_window_inference(test_inputs, cfg.img_size, cfg.sw_batch_size, model)
        pred_all.append(pred)
        # do 4 tta
        for dims in [[2], [3], [2, 3]]:
            flip_pred = sliding_window_inference(torch.flip(test_inputs, dims=dims), cfg.img_size, cfg.sw_batch_size, model)
            flip_pred = torch.flip(flip_pred, dims=dims)
            pred_all.append(flip_pred)
    pred_all = torch.mean(torch.stack(pred_all), dim=0)[0]
    pred_all = post_pred(pred_all)[1:]
    # c, w, h, d to d, c, h, w
    pred_all = torch.permute(pred_all, [3, 0, 2, 1]).cpu().numpy().astype(np.uint8)
    id_outputs3 = from_engine(["id"])(batch)[0]

    for test_output3, id_output3 in zip(pred_all, id_outputs3):
        id_name = id_output3[0]
        lb, sb, st = test_output3
        outputs3.append([id_name, "large_bowel", lb])
        outputs3.append([id_name, "small_bowel", sb])
        outputs3.append([id_name, "stomach", st])

100%|██████████| 7/7 [00:15<00:00,  2.15s/it]


In [56]:
submit3 = pd.DataFrame(data=np.array(outputs3), columns=["id", "class3", "predicted3"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [27]:
class cfg:
    img_size = (224, 224, 80)
    in_channels = 1
    out_channels = 4
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    weights = glob("../input/uw3dweights/unet_fold0_0.9108.pth")
    batch_size = 1
    sw_batch_size = 4

In [28]:
test_transforms = Compose(
    [
        LoadImaged(keys="image"), # d, h, w
        AddChanneld(keys="image"), # c, d, h, w
        Transposed(keys="image", indices=[0, 2, 3, 1]), # c, w, h, d
        Lambdad(keys="image", func=lambda x: x / x.max()),
#         SpatialPadd(keys="image", spatial_size=cfg.img_size),  # in case less than 80 slices
        EnsureTyped(keys="image", dtype=torch.float32),
    ]
)

test_ds = CacheDataset(
        data=test_data,
        transform=test_transforms,
        cache_rate=0.0,
        num_workers=2,
    )

test_dataloader = DataLoader(
    test_ds,
    batch_size=cfg.batch_size,
    num_workers=2,
    pin_memory=True,
)

In [29]:
model = UNet(
    spatial_dims=3,
    in_channels=cfg.in_channels,
    out_channels=cfg.out_channels,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    kernel_size=3,
    up_kernel_size=3,
    num_res_units=2,
    act="PRELU",
    norm="BATCH",
    dropout=0.2,
    bias=True,
    dimensions=None,
).to(cfg.device)

In [30]:
outputs4 = []

post_pred = Compose([
    Activations(sigmoid=True),
    AsDiscrete(threshold=0.5)
])

model.eval()
torch.set_grad_enabled(False)
progress_bar = tqdm(range(len(test_dataloader)))
val_it = iter(test_dataloader)
for itr in progress_bar:
    batch = next(val_it)
    test_inputs = batch["image"].to(cfg.device)

    pred_all = []
    for weights in cfg.weights:
        model.load_state_dict(torch.load(weights))
        pred = sliding_window_inference(test_inputs, cfg.img_size, cfg.sw_batch_size, model)
        pred_all.append(pred)
        # do 4 tta
        for dims in [[2], [3], [2, 3]]:
            flip_pred = sliding_window_inference(torch.flip(test_inputs, dims=dims), cfg.img_size, cfg.sw_batch_size, model)
            flip_pred = torch.flip(flip_pred, dims=dims)
            pred_all.append(flip_pred)
    pred_all = torch.mean(torch.stack(pred_all), dim=0)[0]
    pred_all = post_pred(pred_all)[1:]
    # c, w, h, d to d, c, h, w
    pred_all = torch.permute(pred_all, [3, 0, 2, 1]).cpu().numpy().astype(np.uint8)
    id_outputs4 = from_engine(["id"])(batch)[0]

    for test_output4, id_output4 in zip(pred_all, id_outputs4):
        id_name = id_output4[0]
        lb, sb, st = test_output4
        outputs4.append([id_name, "large_bowel", lb])
        outputs4.append([id_name, "small_bowel", sb])
        outputs4.append([id_name, "stomach", st])

100%|██████████| 7/7 [00:14<00:00,  2.13s/it]


In [57]:
submit4 = pd.DataFrame(data=np.array(outputs4), columns=["id", "class4", "predicted4"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [32]:
  class cfg:
    img_size = (224, 224, 80)
    in_channels = 1
    out_channels = 4
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    weights = glob("../input/uw3dweights/unet_fold0_new_metric_0.8925.pth")
    batch_size = 1
    sw_batch_size = 4

In [33]:
test_transforms = Compose(
    [
        LoadImaged(keys="image"), # d, h, w
        AddChanneld(keys="image"), # c, d, h, w
        Transposed(keys="image", indices=[0, 2, 3, 1]), # c, w, h, d
        Lambdad(keys="image", func=lambda x: x / x.max()),
#         SpatialPadd(keys="image", spatial_size=cfg.img_size),  # in case less than 80 slices
        EnsureTyped(keys="image", dtype=torch.float32),
    ]
)

test_ds = CacheDataset(
        data=test_data,
        transform=test_transforms,
        cache_rate=0.0,
        num_workers=2,
    )

test_dataloader = DataLoader(
    test_ds,
    batch_size=cfg.batch_size,
    num_workers=2,
    pin_memory=True,
)

In [34]:
model = UNet(
    spatial_dims=3,
    in_channels=cfg.in_channels,
    out_channels=cfg.out_channels,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    kernel_size=3,
    up_kernel_size=3,
    num_res_units=2,
    act="PRELU",
    norm="BATCH",
    dropout=0.2,
    bias=True,
    dimensions=None,
).to(cfg.device)

In [35]:
outputs5 = []

post_pred = Compose([
    Activations(sigmoid=True),
    AsDiscrete(threshold=0.5)
])

model.eval()
torch.set_grad_enabled(False)
progress_bar = tqdm(range(len(test_dataloader)))
val_it = iter(test_dataloader)
for itr in progress_bar:
    batch = next(val_it)
    test_inputs = batch["image"].to(cfg.device)

    pred_all = []
    for weights in cfg.weights:
        model.load_state_dict(torch.load(weights))
        pred = sliding_window_inference(test_inputs, cfg.img_size, cfg.sw_batch_size, model)
        pred_all.append(pred)
        # do 4 tta
        for dims in [[2], [3], [2, 3]]:
            flip_pred = sliding_window_inference(torch.flip(test_inputs, dims=dims), cfg.img_size, cfg.sw_batch_size, model)
            flip_pred = torch.flip(flip_pred, dims=dims)
            pred_all.append(flip_pred)
    pred_all = torch.mean(torch.stack(pred_all), dim=0)[0]
    pred_all = post_pred(pred_all)[1:]
    # c, w, h, d to d, c, h, w
    pred_all = torch.permute(pred_all, [3, 0, 2, 1]).cpu().numpy().astype(np.uint8)
    id_outputs5 = from_engine(["id"])(batch)[0]

    for test_output5, id_output5 in zip(pred_all, id_outputs5):
        id_name = id_output5[0]
        lb, sb, st = test_output5
        outputs5.append([id_name, "large_bowel", lb])
        outputs5.append([id_name, "small_bowel", sb])
        outputs5.append([id_name, "stomach", st])

100%|██████████| 7/7 [00:14<00:00,  2.12s/it]


In [36]:
lb.shape

(266, 266)

In [37]:
"""a = submit1.iloc[146, :].predicted
for i in range(len(a)):
    s = set(a[i])
    if len(s) > 1:
        print(s)"""

'a = submit1.iloc[146, :].predicted\nfor i in range(len(a)):\n    s = set(a[i])\n    if len(s) > 1:\n        print(s)'

In [38]:
"""import matplotlib.pyplot as plt
plt.imshow(a)"""

'import matplotlib.pyplot as plt\nplt.imshow(a)'

In [39]:
sb.shape

(266, 266)

In [58]:
submit5 = pd.DataFrame(data=np.array(outputs5), columns=["id", "class5", "predicted5"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [41]:
#submit1[submit1["predicted"] != '']

In [60]:
submit2[submit2["predicted2"]!=""]

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


,id,class2,predicted2
0,case123_day0_slice_0001,large_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,case123_day0_slice_0001,small_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,case123_day0_slice_0001,stomach,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,case123_day0_slice_0002,large_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,case123_day0_slice_0002,small_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...
2995,case77_day20_slice_0143,small_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2996,case77_day20_slice_0143,stomach,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2997,case77_day20_slice_0144,large_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2998,case77_day20_slice_0144,small_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [43]:
#submit3.head()

In [61]:
submit4.head()

,id,class4,predicted4
0,case123_day0_slice_0001,large_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,case123_day0_slice_0001,small_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,case123_day0_slice_0001,stomach,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,case123_day0_slice_0002,large_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,case123_day0_slice_0002,small_bowel,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [100]:
submit5.iloc[146]

id                                      case123_day0_slice_0049
class5                                                  stomach
predicted5    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
Name: 146, dtype: object

In [46]:
from scipy import stats

In [63]:
a=np.array([submit1["predicted1"],submit2["predicted2"],submit3["predicted3"],submit4["predicted4"],submit5["predicted5"]])

In [90]:
weight=[0.2,0.2,0.2,0.2,0.2]

In [91]:
weighted_preds=np.tensordot(a,weight,axes=((0),(0)))

In [96]:
pd.DataFrame(weighted_preds)

0

In [101]:
rle_encode(weighted_preds)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  app.launch_new_instance()


'1'

In [103]:
#a = pd.DataFrame(list(a),columns=["predicted"])

In [104]:
#a.iloc[146][0][0]

In [105]:
"""submits = []
for i in range(len(submit2)):
    final_submit = (
    submit2.iloc[i, :].predicted +
    submit4.iloc[i, :].predicted )/ 2.0
     
    new_sub = (final_submit >= 0.5).astype('uint8')
    classes = [submit2.iloc[i, :]["class2"],
               submit4.iloc[i, :]["class4"]]
    new_class = stats.mode(np.array(classes))[0][0]
    
    submits.append({ "id": submit2.iloc[i,:].id, "class": new_class, "predicted": rle_encode(new_sub) })"""

'submits = []\nfor i in range(len(submit2)):\n    final_submit = (\n    submit2.iloc[i, :].predicted +\n    submit4.iloc[i, :].predicted )/ 2.0\n     \n    new_sub = (final_submit >= 0.5).astype(\'uint8\')\n    classes = [submit2.iloc[i, :]["class2"],\n               submit4.iloc[i, :]["class4"]]\n    new_class = stats.mode(np.array(classes))[0][0]\n    \n    submits.append({ "id": submit2.iloc[i,:].id, "class": new_class, "predicted": rle_encode(new_sub) })'

In [ ]:
"""submit_df = pd.DataFrame(submits)
submit_df"""

In [ ]:
#set(final_submit.flatten())

In [ ]:
#submit=pd.concat([submit1,submit2["class2"],submit3["class3"],submit4["class4"],submit5["class5"]],axis=1)

In [ ]:
#submit

In [ ]:
"""from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
submit["class1"]=le.fit_transform(submit["class1"])
submit["class2"]=le.fit_transform(submit["class2"])
submit["class3"]=le.fit_transform(submit["class3"])
submit["class4"]=le.fit_transform(submit["class4"])
submit["class5"]=le.fit_transform(submit["class5"])"""

In [ ]:
#submit

In [ ]:
#from scipy import stats

In [ ]:
#submit.apply(lambda x: stats.mode(np.array([x.class1, x.class2, x.class3, x.class4, x.class5]))[0][0], axis=1)

In [ ]:
#X=submit

In [ ]:
#y=submit["class5"]

In [ ]:
#from lightgbm import LGBMClassifier as lgb

In [ ]:
#X=X.drop(["class5","id","predicted"],axis=1)

In [ ]:
"""from sklearn.model_selection import train_test_split
test_size_pct = 0.20
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size = test_size_pct, random_state = 42)"""

In [ ]:
#y_train

In [ ]:
#model=lgb()

In [ ]:
#model.fit(X,y)

In [ ]:
# Fix sub error, refers to: https://www.kaggle.com/competitions/uw-madison-gi-tract-image-segmentation/discussion/320541
if not debug:
    sub_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
    del sub_df['predicted']
    sub_df = sub_df.merge(submit_df, on=['id','class'])
    sub_df.to_csv('submission.csv',index=False)
else:
    submit_df.to_csv('submission.csv', index=False)